# NTF 6 - Portos
## Esforços Hidrodinâmicos e Momentos sobre pilares e tubulões Portuários

Aluno: Francisco José Matos Nogueira Filho

Matricula: 384962

In [1]:
import numpy as np
from numpy import pi
from IPython.display import Markdown as md

## Dados da onda em Alto Mar

In [2]:
## Dados da onda em Alto Mar

H0 = 4.2 #m
T = 9 #s
g = 9.81 #m/s^2

In [3]:
rhoH2O = 1034 #kgf/m^3
rhoC = 2500 #kgf/m^3
Es = 2.1e6 #kgf/cmˆ2
Ec = 3e5 #kgf/cmˆ2

D = 0.7 #m
As = (pi * (1.905e-2)**2/4)* 16
Ac = pi*D**2/4
l = 25 #m

### a) A altura da onda a atingir a zona de cravação das estacas

In [4]:
def comprimentoDeOnda(Periodo):
    return (g * Periodo**2)/(2 * pi)

L0 = comprimentoDeOnda(T)

#Resposta
md("$L_0 = %.2f \; m$"%(L0))

$L_0 = 126.47 \; m$

In [5]:
def comprimentoDeOnda(Periodo):
    return (g * Periodo**2)/(2 * pi)
def Celeridade(Comprimento,T):
    return Comprimento/T
L0 = comprimentoDeOnda(T)
C0 = Celeridade(L0,T)
sigma = 2*pi/T
#Resposta
md("""$L_0 = %.2f \; m$ \n
$C_0 = %.2f \; m/s$"""%(L0,C0))

$L_0 = 126.47 \; m$ 

$C_0 = 14.05 \; m/s$

In [6]:
from scipy.optimize import broyden1
#from scipy.optimize import minimize, newton_krylov, broyden1, curve_fit
d1 = 12 #m
def fu(l):
    return (l/L0 - np.tanh((2*pi*d1)/l))**2
x0 = 10 #Palpite inicial
solv = broyden1(fu,x0,iter=50)
L12 = solv

def calcularK(L):
    return (2 * pi)/L

k12 = calcularK(L12)
md("""
$L = %.2f \; m $ \n
$K = %.5f \; $
""" % (L12,k12))


$L = 87.90 \; m $ 

$K = 0.07148 \; $


In [7]:
def calcularN(K,d):
    return 0.5 * (1 + (2*K*d)/np.sinh(2*K*d))

n12 = calcularN(k12,d1)
md("""
$n = %.3f$
""" % n12)


$n = 0.819$


# $ h = H_0 \sqrt{\frac{b_0}{b}}\sqrt{\frac{1}{2} \frac{1}{n} \frac{C_0}{C}} $
Pode-se considerar $ \frac{b_0}{b} = 0$ uma vez que se estamos calculando para um cenário em Mar Aberto

In [8]:
def calcularH(H0,n,C0,C):
    return H0 * (1 * 0.5* 1/n * C0/C)**0.5
def VelocidadeOrbitalMax(a,K,sigma):
    return (a * g * K)/sigma
C12 = Celeridade(L12,T)
H12 = calcularH(H0,n12,C0,C12)
a12 = H12/2
u12 = VelocidadeOrbitalMax(a12,k12,sigma)
md("""
$H = %.3f \; m $ \n
$u = %.3f \; m/s$
""" % (H12, u12))


$H = 3.936 \; m $ 

$u = 1.977 \; m/s$


### b) A máxima força de inércia provocada pela onda ao passar pela estaca
# $$ F_M = C_M  \rho g \frac{\pi D^2}{4} H k_m $$
# $$ k_m = \frac{1}{2} \tanh(Kd) \; sen \left(\frac{-2 \pi t}{T} \right)$$ 

Para $ sen(\frac{-2 \pi t}{T}) = 1$ tem-se $ k_d = \frac{1}{2}\tanh(Kd)$

In [9]:
Km = 0.5 * np.tanh(k12*d1)
Cm = 1.4
Fm = Cm * rhoH2O * g * (pi * D**2)/4 * H12 * Km
md("""
$F_M = %.3f \; tf$
""" % (Fm/9.81e3))


$F_M = 0.762 \; tf$


### c) A máxima força de arraste provocada pela onda ao passar pela estaca (0,1 ponto);
# $$ F_D = C_D \frac{1}{2} \rho g D H^2 k_d $$
# $$ k_d = \frac{1}{4} n \left| \cos(\frac{2 \pi t}{T}) \right| \cos(\frac{2 \pi t}{T})$$ 

Para $ \cos(\frac{2 \pi t}{T}) = 1$ tem-se $ K_d = \frac{n}{4} $

In [10]:
Kd = n12/4
Cd = 1.05
Fd = Cd * 0.5 * rhoH2O * g * D * H12**2 * Kd
md("""
$F_D = %.3f \; tf$
""" % (Fd/9.81e3))


$F_D = 1.205 \; tf$


### d) O máximo momento de inércia no pé da estaca provocado pela onda
# $$ M_M = F_M \; d \; s_m $$
# $$ s_m = 1 + \frac{1 - \cosh(Kd)}{K d \; senh( K d)} $$

In [11]:
sm = 1 + (1 - np.cosh(k12*d1))/(k12*d1 *np.sinh(k12*d1))
Mm = Fm * d1 * sm
md("""
$M_M = %.3f \; tf•m$
""" % (Mm/9.81e3))


$M_M = 4.834 \; tf•m$


### e) O máximo momento de arraste no pé da estaca provocado pela onda
# $$ M_d = F_d \; d \; s_d $$
# $$ s_d = \frac{1}{2}+ \frac{1}{2n} \left(\frac{1}{2} + \frac{1 - \cosh(2Kd)}{2 K d \; senh(2 K d)}  \right) $$

In [12]:
sd = 0.5 + 1/(2*n12) * (0.5 + (1 - np.cosh(2* k12*d1))/(2 * k12*d1 * np.sinh(2 * k12*d1)))
Md = Fd * d1 * sd
md("""
$M_D = %.3f \; tf•m$
""" % (Md/9.81e3))


$M_D = 8.070 \; tf•m$


### f) A resultante combinada dos momentos de inércia e arraste no pé da estaca
# $$ M = M_D + M_M $$

In [13]:
M = Md + Mm
md("""
$M = %.3f \; tf•m$
""" % (M/9.81e3))


$M = 12.904 \; tf•m$


### g) A frequência de vibração própria da estaca numa condição de simplesmente cravada

In [14]:
J = (pi*D**4)/64
md("""$J = %.4f m^4$""" %J)


$J = 0.0118 m^4$

In [15]:
Ep = (As *Es + Ac*Ec)/(As + Ac)
md("""$E_p = %.2f kgf/cm^2$""" %Ep)

$E_p = 321079.98 kgf/cm^2$

In [16]:
_m = (rhoH2O + rhoC) * pi * D**2/4
md("""$\overline{m} = %.2f kg/m^3/ml$""" %_m)

$\overline{m} = 1360.04 kg/m^3/ml$

In [17]:
fator = (Ep*1e4*J/_m)**0.5
md(r"""
$\sqrt{\frac{E J}{\overline{m}}} = %.2f$
""" %fator)


$\sqrt{\frac{E J}{\overline{m}}} = 166.81$


In [18]:
k = 0.56
f1 = k/l**2 * fator
md("""$f_1 = %.5f /seg$""" %f1)

$f_1 = 0.14946 /seg$

### h) A velocidade crítica de corrente capaz de produzir ressonância com a estaca simplesmente cravada

In [19]:
S = 0.2
U0 = (f1*D)/S
md("""$U_0 = %.5f m/s$""" %U0)

$U_0 = 0.52310 m/s$

### i) O valor da força transversal máxima L provocada pela corrente, considerando que $sen(2 \pi f_k t) = 1 $.
# $$ L = c_K \frac{1}{2} \rho U_0^2 D \; sen(2 \pi f_k t) = 1 $$

In [20]:
Ck = 0.2
Vcorr = 6 #nós
Vcorr_ = Vcorr* 0.5144
L = Ck * 0.5 * rhoH2O * U0**2 * D
md("""$L = %.2f Kgf$""" %L)

$L = 19.81 Kgf$

In [21]:
if U0 > Vcorr_:
    print("Risco de haver Ressonância")
else:
    print("Sem risco de haver Ressonância")

Sem risco de haver Ressonância
